This tutorial assumes that the HyDrop fastq files have been placed in a directory `fastq/`.

In [8]:
ls fastq/HYA*R?.fastq.gz

fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-1_S1_R1.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-1_S1_R2.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-1_S1_R3.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-2_S2_R1.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-2_S2_R2.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-2_S2_R3.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-3_S3_R1.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-3_S3_R2.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-3_S3_R3.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-4_S4_R1.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-4_S4_R2.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-4_S4_R3.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-5_S5_R1.fastq.gz
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-5_S5_R2.fastq.gz
fastq/HYA__combined__20210323_cort

A hydrop-rna library looks like this:

```
               Index 1: i7 read 10 bp           Read 1: cDNA read 86 bp (R1)

             <- HyDrop_CustSeq_Short_i7_Read            <- NXT               
|-5’-[P7]-[i7]-[HyDrop_CustSeq_Short]-[BC_UMI]-[polyA]-[cDNA]-[NXT]-[i5]-[P5]-3’
  3’-[P7]-[i7]-[HyDrop_CustSeq_Short]-[BC_UMI]-[polyT]-[cDNA]-[NXT]-[i5]-[P5]-5’-|  
                HyDrop_CustSeq_Short ->                    NXT ->    
                
                   Read 2: BC 60 bp  (R2)           Index 2: i5 read 10 bp       

```

And a HyDrop-ATAC library looks like so:

```
                                Index 1: BC read  (R2)     Read 1: ATAC mate 1  (R1)

                                         <- Tn5       <- Tn5               
|-5’-[P7]-[i7]-[HyDrop_CustSeq_Short]-[BC]-[Tn5]-[gDNA]-[Tn5]-[i5]-[P5]-3’
  3’-[P7]-[i7]-[HyDrop_CustSeq_Short]-[BC]-[Tn5]-[gDNA]-[Tn5]-[i5]-[P5]-5’-|  
                                            Tn5 ->       Tn5 ->    
                
                               Read 2: ATAC mate 2 (R3)   Index 2: i5 read 10 bp       

```

The barcode read is ultimately structured like this for both protocols:

```
-[BC1]-[AD1]-[BC2]-[AD2]-[BC3]-[UMI]
-[10b]-[10b]-[10b]-[10b]-[10b]-[8bp]
```

With the lengths of each part under the part.
The adapters inbetween the 3 sub-barcodes are identical for all reads.
We must cut those out first and write a "BCONLY" (barcode only) fastq file.
We use (M)AWK for this.

In [2]:
module load mawk

split_fastq (){
    local input_fastq_filename="${1}";
    local output_fastq_split1_filename="${2}";
    local split1_start=1;
    local split1_end=10;
    local split2_start=21;
    local split2_end=30;
    local split3_start=41;
    local split3_end=50;
    zcat "${input_fastq_filename}" \
      | mawk \
            -v "split1_start=${split1_start}" \
            -v "split1_end=${split1_end}" \
            -v "split2_start=${split2_start}" \
            -v "split2_end=${split2_end}" \
            -v "split3_start=${split3_start}" \
            -v "split3_end=${split3_end}" \
            -v "output_fastq_split1_filename=${output_fastq_split1_filename}" \
            '
            BEGIN {
                split1_length = split1_end - split1_start + 1;
                split2_length = split2_end - split2_start + 1;
                split3_length = split3_end - split3_start + 1;
            }
            {
                if (NR % 2 == 1) {
                    # Read name or "+" line.
                    print $0 > output_fastq_split1_filename
                } else {
                    # Sequence or quality line.
                    print substr($0, split1_start, split1_length) substr($0, split2_start, split2_length) substr($0, split3_start, split3_length) > output_fastq_split1_filename;
                }
            }'
}

In [46]:
for fastq in fastq/*combined*S?_R2.fastq.gz
do
    newname=${fastq%.fastq.gz}.BCONLY.fastq
    echo $newname
    split_fastq $fastq $newname &
done

fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-1_S1_R2.BCONLY.fastq
[11] 3909
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-2_S2_R2.BCONLY.fastq
[12] 3910
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-3_S3_R2.BCONLY.fastq
[13] 3911
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-4_S4_R2.BCONLY.fastq
[14] 3914
fastq/HYA__combined__20210323_cortex_phu_dv_etssb_1-5_S5_R2.BCONLY.fastq
[15] 3917


Compress the resulting BCONLY fastq files. You can also pipe the MAWK output directly to gzip or pigz, but I often like to inspect the fastq files before proceeding.

In [56]:
module load pigz
for fastq in fastq/*BCONLY.fastq
do
    pigz -p 6 $fastq &
done

[1] 4547
[2] 4548
[3] 4549
[4] 4550
[5] 4551


Now, we use the VSN pipeline to perform barcode correction, map to a reference genome and generate fragments files.
First, we must make a metadata file as described in the VSN documentation. The script below automates this step so no copy/pasting errors are made.
Make sure that you use the bconly files in vsn pipeline.

In [62]:
rm metadata_auto.tsv
dir=`pwd -P`
echo -e 'sample_name\ttechnology\tfastq_PE1_path\tfastq_barcode_path\tfastq_PE2_path' > metadata_auto.tsv
for fastq in fastq/*combined*S?_R2.fastq.gz
do
    samplename=${fastq%_R2.fastq.gz}
    metadatasamplename=${samplename#fastq/}
    R1=$dir/${samplename}_R1.fastq.gz
    R2=$dir/${samplename}_R2.BCONLY.fastq.gz
    R3=$dir/${samplename}_R3.fastq.gz
    echo -e $metadatasamplename'\tstandard\t'$R1'\t'$R2'\t'$R3 >> metadata_auto.tsv
done

Then, we must generate a config file to configure the VSN pipeline parameters.

In [9]:
module load graphviz
module load Nextflow
nextflow pull vib-singlecell-nf/vsn-pipelines -r develop_atac

nextflow config \
    vib-singlecell-nf/vsn-pipelines/main_atac.nf \
    -profile atac_preprocess,singularity \
    > atac_preprocess.config


The following have been reloaded with a version change:
  1) Java/1.8.0_162 => Java/1.8.0

Checking vib-singlecell-nf/vsn-pipelines ...
 Already-up-to-date - revision: 68397367f7 [develop_atac]


Now, we must make some important changes to the config file:
* Redirect the config file to metadata_auto.tsv instead of standard metadata.tsv
* If you use multispecies bwa index, make sure to edit the sinto regex for which fragments to accept! Try with '"(?i)^GRCh38_chr|(?i)^mm10"'. Generally, this regex should allow all the chromosomes which you want to be included in the final fragments file.
* Change the bwa executor to local if you want the job to start immediately. Otherwise, you can keep the PBS job submission system in place.
* Number of CPUs for BWA: better to have 2 forks running with 17 threads than to have 1 fork with 36 threads due to I/O overhead
* Change the bwa index directory to the right one!
* Add whitelist path under 'standard'

Now, run in a new tmux session (to prevent interruption):

In [ ]:
nextflow -C atac_preprocess.config run \
    vib-singlecell-nf/vsn-pipelines/main_atac.nf \
    -entry atac_preprocess -r develop_atac -resume

After the run has finished, copy the fragments files to a new dir `fragments/`. This ensures that if you accidentally modify or delete the fragments files, you still have a backup and don't need to re-run the fragments file generation step.

In [63]:
mkdir fragments/
cp out/data/fragments/* fragments/
cp out/data/bam/*mapping_stats.tsv fragments/
cp out/data/fastq/*.corrected.bc_stats.tsv fragments/

mkdir: cannot create directory ‘fragments/’: File exists


I also like uncompressed copies of the fragments files at hand for easy inspection and modification.

In [64]:
for file in fragments/*.gz
do
    newname=${file%.gz}
    echo $newname
    /vsc-hard-mounts/leuven-data/software/biomed/skylake_centos7/2018a/software/zlib-ng/2.0.1-foss-2018a/bin/zlib_ng.sh zcat $file > $newname &
done

fragments/HYA__932b0e__20210309_mouse_cortex_MCF-7_PC3_1-1_S2.sinto.fragments.NONEGS.tsv
[1] 36310
fragments/HYA__932b0e__20210309_mouse_cortex_MCF-7_PC3_1-1_S2.sinto.fragments.tsv
[2] 36311
fragments/HYA__af8d60__20210309_mouse_cortex_MCF-7_PC3_1-2_S3.sinto.fragments.NONEGS.tsv
[3] 36312
fragments/HYA__af8d60__20210309_mouse_cortex_MCF-7_PC3_1-2_S3.sinto.fragments.tsv
[4] 36313
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-1_S1.sinto.fragments.tsv
[5] 36314
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-2_S2.sinto.fragments.tsv
[6] 36315
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-3_S3.sinto.fragments.tsv
[7] 36316
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-4_S4.sinto.fragments.tsv
[8] 36317
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-5_S5.sinto.fragments.tsv
[9] 36318


Sometimes, especially on chrM, sinto can generate negative coordinates for fragments close to the edge of the chromosome. This can give problems with some tools which use fragments files. Therefore, we replace all negative start positions by zeroes.

In [67]:
module load mawk
for tsv in fragments/*.sinto.fragments.tsv
do
    echo $tsv
    newname=${tsv%.tsv}.NONEGS.tsv
    mawk -v OFS='\t' -F '\t' '$2 ~ /^-/ {$2=1}1' $tsv > $newname &
done

fragments/HYA__932b0e__20210309_mouse_cortex_MCF-7_PC3_1-1_S2.sinto.fragments.tsv
[1] 36508
fragments/HYA__af8d60__20210309_mouse_cortex_MCF-7_PC3_1-2_S3.sinto.fragments.tsv
[2] 36509
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-1_S1.sinto.fragments.tsv
[3] 36510
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-2_S2.sinto.fragments.tsv
[4] 36511
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-3_S3.sinto.fragments.tsv
[5] 36512
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-4_S4.sinto.fragments.tsv
[6] 36513
fragments/HYA__combined__20210323_cortex_phu_dv_etssb_1-5_S5.sinto.fragments.tsv
[7] 36514


Bgzip the uncompressed `NONEGS` fragments files. As stated before, you can pipe these commands, but I like to inspect inbetween steps.

In [69]:
module load BCFtools
for tsv in fragments/*.sinto.fragments.NONEGS.tsv
do
    bgzip -@ 6 $tsv &
done


The following have been reloaded with a version change:
  1) XZ/5.2.3-GCCcore-6.4.0 => XZ/5.2.4-GCCcore-6.4.0
  2) bzip2/1.0.6-GCCcore-6.4.0 => bzip2/1.0.8-GCCcore-6.4.0

[1] 36784
[2] 36785
[3] 36786
[4] 36787
[5] 36788
[6] 36789
[7] 36790


If you have species-mixed data, you can already separate fragments mapping to either species from the other. This of course depends on the index (chromosome names) that you used to map. Below, is a command I use if I want to do this for mouse and human data. Since we are currently analysing mouse data only, it is not necessary. In fact, it won't do anything as none of the chromosome names start with `mm10`.

In [36]:
module loads BCFtools
for frags in fragments/*.tsv.gz
do
    # echo $frags
    newname=${frags%.sinto.fragments.tsv.gz}
    echo $newname
    zgrep ^mm10 $frags | sed 's/^mm10_//g' | bgzip -c > $newname.sinto.fragments.MM10.gz &
    zgrep ^GRCh38 $frags | sed 's/^GRCh38_//g' | bgzip -c > $newname.sinto.fragments.GRCH38.gz &
done

fragments/HYA__665d2b__20210323_cortex_phu_dv_etssb_1-1_S1
[1] 35032
[2] 35035
fragments/HYA__8aaaff__20210323_cortex_phu_dv_etssb_1-2_S2
[3] 35038
[4] 35041
fragments/HYA__932b0e__20210309_mouse_cortex_MCF-7_PC3_1-1_S2
[5] 35044
[6] 35047
fragments/HYA__a64416__20210323_cortex_phu_dv_etssb_1-3_S3
[7] 35050
[8] 35053
fragments/HYA__af8d60__20210309_mouse_cortex_MCF-7_PC3_1-2_S3
[9] 35056
[10] 35059
fragments/HYA__b8797a__20210323_cortex_phu_dv_etssb_1-4_S4
[11] 35062
[12] 35065
fragments/HYA__f1116e__20210323_cortex_phu_dv_etssb_1-5_S5
[13] 35068
[14] 35071
